### CURRENCY CONVERSION TOOL

In [36]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from langchain_core.tools import tool
from dotenv import load_dotenv
from langchain_core.tools import InjectedToolArg
from typing import Annotated
import requests

### Tool Creation

In [37]:
@tool 
def get_conversion_factor(base_currency: str, target_currency:str) -> float:
    """
    This function fetches the currency conversion factor between a given base currency and a target currency.
    """
    url = f'https://v6.exchangerate-api.com/v6/222e06b00a690c2392706798/pair/{base_currency}/{target_currency}'

    response = requests.get(url)

    return response.json()


@tool
def convert_currency(base_currency_value: float, conversion_rate: Annotated[float, InjectedToolArg]) -> float:
    """
    Given a currency conversion rate this function calculates the target currency value from a given base currency value.
    """
    return base_currency_value * conversion_rate




In [38]:
# trying the tool
get_conversion_factor({'base_currency':'USD','target_currency':'INR'})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1746835202,
 'time_last_update_utc': 'Sat, 10 May 2025 00:00:02 +0000',
 'time_next_update_unix': 1746921602,
 'time_next_update_utc': 'Sun, 11 May 2025 00:00:02 +0000',
 'base_code': 'USD',
 'target_code': 'INR',
 'conversion_rate': 85.4999}

In [39]:
convert_currency({'base_currency_value':100,'conversion_rate':82.5})

8250.0

In [40]:
llm = ChatOpenAI()

In [41]:
#tool binding
llm_with_tools = llm.bind_tools([get_conversion_factor, convert_currency])

In [42]:
messages = [HumanMessage(content="What is the conversion rate of USD and INR, and based on that can you convert 10 usd to inr?")]

In [43]:
AI_message = llm_with_tools.invoke(messages)
messages.append(AI_message)

In [44]:
import json

for tool_call in AI_message.tool_calls:
    if tool_call['name'] == 'get_conversion_factor':
        #Invoke the tool to get tool_message which contains the conversion rate
        tool_message1 = get_conversion_factor.invoke(tool_call)
        #Extract the conversion rate from the tool_message
        conversion_rate = json.loads(tool_message1.content)['conversion_rate']
        messages.append(tool_message1)

    if tool_call['name'] == 'convert_currency':
        #fetch the current arg
        tool_call['args']['conversion_rate'] = conversion_rate
        tool_message2 = convert_currency.invoke(tool_call)
        messages.append(tool_message2)



In [45]:
messages

[HumanMessage(content='What is the conversion rate of USD and INR, and based on that can you convert 10 usd to inr?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_J1xPQXquPcz5ls2ejP7R81Mh', 'function': {'arguments': '{"base_currency": "USD", "target_currency": "INR"}', 'name': 'get_conversion_factor'}, 'type': 'function'}, {'id': 'call_cOs9hUdjc6eUkXybEwz35ySK', 'function': {'arguments': '{"base_currency_value": 10}', 'name': 'convert_currency'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 54, 'prompt_tokens': 125, 'total_tokens': 179, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BVelSHXjz7lZMmrvVqTQqPfQQMo0D', 'service_tier': 'd

In [46]:
llm_with_tools.invoke(messages).content

'The conversion rate of USD to INR is 1 USD = 85.4999 INR. \nTherefore, 10 USD is equal to 854.999 INR.'